리트코드 큐를 이용한 스택 구현: https://leetcode.com/problems/implement-stack-using-queues/

In [1]:
import collections
import heapq
import functools
import itertools
import re
import sys
import math
import bisect
from typing import *

# 큐 Queue

큐란 시퀀스의 한쪽 끝에는 엔티티를 추가하고, 반대쪽 끝에서는 제거하는 엔티티 컬렉션이다. FIFO(First-In-First-Out)으로 처리되며 줄 서는 것에 비유된다.

Deque나 우선순위 큐 같은 변형이 존재하며 BFS나 캐시 등을 구현할 때도 사용된다.

파이썬의 queue 모듈은 자료구조로서의 큐보다는 동기화 기능에 집중된 모듈로 큐 자료구조를 구현할 때는 큐의 모든 연산을 가진 파이썬 리스트나 양방향 삽입 삭제에 O(1)의 복잡도를 갖는 deque를 사용한다고 한다.

# push()할 때 큐를 이용해 재정렬

In [2]:
class MyStack:

    def __init__(self):
        self.q = collections.deque() # deque를 이용해 큐 자료구조 사용
        

    def push(self, x: int) -> None:
        self.q.append(x)
        # 요소를 재정렬
        for _ in range(len(self.q) - 1):
            self.q.append(self.q.popleft())

    def pop(self) -> int:
        return self.q.popleft()

    def top(self) -> int:
        return self.q[0]

    def empty(self) -> bool:
        return len(self.q) == 0

In [3]:
obj = MyStack()

In [4]:
obj.push(1)

In [5]:
obj.push(2)

In [6]:
obj.top()

2

In [7]:
obj.pop()

2

In [8]:
obj.empty()

False

deque를 큐로 이용하여 스택을 구현했다. <br>
push 연산을 구현할 때 큐에 재정렬을 가해주면 스택을 쉽게 구현할 수 있다. <br>
책에서는 큐의 연산만을 사용하여 구현할 것이라고 하는데 책에서 쓴 방법도 popleft가 들어가 결국 deque에 맞춰진 것 아닌가 싶었다. pop만 사용하거나 직접 queue를 구현해본 결과 저자가 의도한 것은 결국 한 방향으로만 들어왔다 나갔다 하는 것으로 설정한 방향의 차이인 것을 깨달았다.

## pop으로 써보기

In [9]:
# pop만 쓰기
class MyStack:

    def __init__(self):
        self.q = collections.deque() # deque를 이용해 큐 자료구조 사용
        

    def push(self, x: int) -> None:
        self.q.append(x)
        # 큐에서는 맨 뒤로 요소가 들어가므로 스택에 맞게 맨 앞으로 재정렬
        for _ in range(len(self.q) - 1):
            self.q.append(self.q.pop())

    def pop(self) -> int:
        return self.q.pop()

    def top(self) -> int:
        return self.q[len(self.q) - 1]

    def empty(self) -> bool:
        return len(self.q) == 0

In [10]:
obj = MyStack()

In [11]:
obj.push(1)

In [12]:
obj.push(2)

In [13]:
obj.top()

2

In [14]:
obj.pop()

2

In [15]:
obj.empty()

False

사실 이게 원래 큐 자료구조가 나가는 방향이긴 하다. 

## 직접 큐 구현하기

In [16]:
from typing import Any


class Queue:
    
    def __init__(self, *args: Any):
        self.__items: list[Any] = list(args)
        self.__size: int = len(self.__items)
        
    def enqueue(self, item: Any):
        self.__items.append(item)
        self.__size += 1
    
    def dequeue(self):
        if self.empty():
            raise IndexError("dequeue from empty queue")
        item: Any = self.__items.pop(0)
        self.__size -= 1
        return item

    def empty(self):
        return not self.__size
    
    def peek(self):
        if self.empty():
            raise IndexError("dequeue from empty queue")
        item: Any = self.__items[0]
        return item
    
    def __str__(self):
        return f"{self.__items}"
    
    def __repr__(self):
        return f"{self.__items}"
    
    def __len__(self):
        return self.__size
    
    def __iter__(self):
        if self.empty():
            return None
        for item in self.__items:
            yield item
            
class MyStack:

    def __init__(self):
        self.q = Queue()
        

    def push(self, x: int) -> None:
        self.q.enqueue(x)
        # 큐에서는 맨 앞으로 요소가 들어가므로 pop으로 꺼낼 수 있게 맨 뒤로 재정렬
        for _ in range(len(self.q) - 1):
            self.q.enqueue(self.q.dequeue())

    def pop(self) -> int:
        return self.q.dequeue()

    def top(self) -> int:
        return self.q.peek()

    def empty(self) -> bool:
        return self.q.empty()

In [17]:
obj = MyStack()

In [18]:
obj.push(1)

In [19]:
obj.push(2)

In [20]:
obj.top()

2

In [21]:
obj.pop()

2

In [22]:
obj.empty()

False

나무위키에 있는 큐를 구현하여 그것을 이용해 스택을 구현해 보았다. peek 연산이 없길래 직접 구현했다. 큐는 리스트를 사용해 구현되었다. <br>
이 풀이를 풀면서 이 알고리즘의 문제를 깨달았는데 파이썬이라서 리스트를 사용했기 때문에 큐 하나를 가지고 재정렬 작업을 할 수 있었다. 그렇지만 실제로 c나 자바 등의 배열을 사용했다면 이 풀이는 불가능하다. 해당 구조를 통해 큐를 구현하면 재정렬 과정에서 양끝을 가리키는 포인터가 만나 큐의 전체적인 용량이 줄어들어 0이 되기 때문이다. 그 언어들로 자료구조들을 배울 때 이 문제의 해결법으로 순환큐를 배웠던 기억이 난다. <br>
그래서 다른 언어 풀이를 검색 해보니 사람들은 두 개의 큐 자료구조를 사용하여 그 문제를 해결하였다.